# Benchmarking pip vs uv

**Ziel:**
Vergleiche die Installationsgeschwindigkeit und Ressourcennutzung von `pip install` vs. `uv pip sync` in einem kleinen Beispielprojekt.

**Abhängigkeiten:** numpy, pandas, click, pytest

In [8]:
import sys, os, subprocess, shutil, time
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Installiere fehlende Bibliotheken automatisch
required = ["pandas", "matplotlib"]
for pkg in required:
    try:
        __import__(pkg)
    except ImportError:
        subprocess.run([sys.executable, "-m", "pip", "install", pkg], check=True)

In [9]:
# Zeitmessung für Befehle
def time_cmd(cmd):
    start = time.perf_counter()
    subprocess.run(cmd, check=True)
    return time.perf_counter() - start

# Venv-Pfad je nach Tool
def venv_path(tool):
    return Path(f"venv-{tool}")

# Erstinstallation mit pip oder uv
def install_venv(tool):
    venv = venv_path(tool)
    if venv.exists():
        shutil.rmtree(venv)
    # Venv anlegen
    subprocess.run([sys.executable, '-m', 'venv', str(venv)], check=True)
    pip_exe = venv / ("Scripts/pip.exe" if os.name=='nt' else "bin/pip")
    if tool == 'uv':
        # uv in venv installieren
        subprocess.run([str(pip_exe), 'install', 'uv'], check=True)
    return pip_exe

In [10]:
results = []
iters = 3  # Anzahl Wiederholungen für jede Messung
for mode in ['cold', 'warm']:
    for tool in ['pip', 'uv']:
        times = []
        for _ in range(iters):
            pip_exe = install_venv(tool)
            # Je nachdem, ob cold oder warm
            if mode == 'warm':
                # Einmal vorinstallieren (warmes Szenario)
                subprocess.run([str(pip_exe), 'install', '-r', 'requirements.txt'], check=True)
            # Jetzt messen
            if tool == 'pip':
                cmd = [str(pip_exe), 'install', '-r', 'requirements.txt']
            else:
                uv_exe = pip_exe.parent / ('uv.exe' if os.name=='nt' else 'uv')
                cmd = [str(uv_exe), 'pip', 'sync', 'requirements.txt']
            t = time_cmd(cmd)
            times.append(t)
        # Ergebnisse aggregieren
        results.append({
            'tool': tool,
            'mode': mode,
            'mean_time_s': sum(times)/len(times),
            'std_time_s': (sum((x-(sum(times)/len(times)))**2 for x in times)/len(times))**0.5
        })

df = pd.DataFrame(results)
df

  Using cached numpy-2.2.5-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached click-8.2.0-py3-none-any.whl.metadata (2.5 kB)
  Using cached pytest-8.3.5-py3-none-any.whl.metadata (7.6 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached iniconfig-2.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pluggy-1.6.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached numpy-2.2.5-cp313-cp313-macosx_14_0_arm64.whl (5.1 MB)
Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl (11.3 MB)
Using cached click-8.2.0-py3-none-any.whl (102 kB)
Using cached pytest-8.3.5-py3-none-any.whl (343 kB)
Usi


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python3.13 -m pip install --upgrade pip


  Using cached numpy-2.2.5-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached click-8.2.0-py3-none-any.whl.metadata (2.5 kB)
  Using cached pytest-8.3.5-py3-none-any.whl.metadata (7.6 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached iniconfig-2.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pluggy-1.6.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached numpy-2.2.5-cp313-cp313-macosx_14_0_arm64.whl (5.1 MB)
Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl (11.3 MB)
Using cached click-8.2.0-py3-none-any.whl (102 kB)
Using cached pytest-8.3.5-py3-none-any.whl (343 kB)
Usi


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python3.13 -m pip install --upgrade pip


  Using cached numpy-2.2.5-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached click-8.2.0-py3-none-any.whl.metadata (2.5 kB)
  Using cached pytest-8.3.5-py3-none-any.whl.metadata (7.6 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached iniconfig-2.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pluggy-1.6.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached numpy-2.2.5-cp313-cp313-macosx_14_0_arm64.whl (5.1 MB)
Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl (11.3 MB)
Using cached click-8.2.0-py3-none-any.whl (102 kB)
Using cached pytest-8.3.5-py3-none-any.whl (343 kB)
Usi

...

  Using cached uv-0.7.5-py3-none-macosx_11_0_arm64.whl.metadata (11 kB)
Using cached uv-0.7.5-py3-none-macosx_11_0_arm64.whl (15.6 MB)



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python3.13 -m pip install --upgrade pip


  Using cached numpy-2.2.5-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached click-8.2.0-py3-none-any.whl.metadata (2.5 kB)
  Using cached pytest-8.3.5-py3-none-any.whl.metadata (7.6 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached iniconfig-2.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pluggy-1.6.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached numpy-2.2.5-cp313-cp313-macosx_14_0_arm64.whl (5.1 MB)
Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl (11.3 MB)
Using cached click-8.2.0-py3-none-any.whl (102 kB)
Using cached pytest-8.3.5-py3-none-any.whl (343 kB)
Usi


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python3.13 -m pip install --upgrade pip
Resolved 4 packages in 3ms
Audited 4 packages in 0.21ms


,tool,mode,mean_time_s,std_time_s
0,pip,cold,6.068932,0.126056
1,uv,cold,0.383258,0.007354
2,pip,warm,0.309764,0.006515
3,uv,warm,0.438231,0.015279


In [11]:
sizes = []
for tool in ['pip','uv']:
    venv = venv_path(tool)
    if venv.exists():
        usage = shutil.disk_usage(venv)
        sizes.append({'tool': tool, 'size_mb': usage.used/1024**2})

df_sizes = pd.DataFrame(sizes)
df_sizes

,tool,size_mb
0,pip,196421.152344
1,uv,196421.152344


In [16]:
import matplotlib
# Nutze das nicht-interaktive Agg-Backend, das überall funktioniert
matplotlib.use('Agg')

import matplotlib.pyplot as plt
from IPython.display import display

# Balkendiagramm: Installationszeiten
fig, ax = plt.subplots()
for mode in ['cold', 'warm']:
    subset = df[df['mode'] == mode]
    ax.bar(
        subset['tool'] + f' ({mode})',
        subset['mean_time_s'],
        yerr=subset['std_time_s'],
        capsize=5
    )
ax.set_ylabel('Durchschnittliche Zeit (s)')
ax.set_title('pip vs uv: Installationszeiten')
plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
plt.tight_layout()

display(fig)
plt.close(fig)

# Balkendiagramm: Festplattennutzung
fig2, ax2 = plt.subplots()
ax2.bar(df_sizes['tool'], df_sizes['size_mb'])
ax2.set_ylabel('Genutzter Speicher (MB)')
ax2.set_title('pip vs uv: Festplattennutzung')
plt.tight_layout()

display(fig2)
plt.close(fig2)


ModuleNotFoundError: No module named 'matplotlib.backends.backend_agg'